In [ ]:
!pip install awswrangler

In [ ]:
import awswrangler as wr
from datetime import datetime, timedelta 
import logging
import pandas as pd 
from pathlib import Path as Path
import time
import urllib3
import warnings

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore")
http = urllib3.PoolManager(num_pools = 50)

In [ ]:
helper_function_path = Path.cwd().parent / 'etl_helper/helper_function.py'
%run {helper_function_path}

In [ ]:
#Trigger pre-pivot sql
current_path = str(Path.cwd())
current_path = current_path.replace('notebooks', 'sql')
final_path_pre = Path(current_path + '/mars_pre_pivot.sql')
trigger_all_functions([final_path_pre])

In [ ]:
#Pivot table
target_database = 'sandbox_measurement_analyst'
target_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'
output_table_name = 'mars_post_pivot_20240526_staging'
output_path = f's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_pivoted_outputs/final/'
today = datetime.today().strftime('%Y%m%d')

def generate_pivoted(target_database, target_workgroup, output_table_name, output_path, today):
    df_all = wr.athena.read_sql_query(
        f"""SELECT * FROM sandbox_measurement_analyst.mars_pre_pivot_20240526_staging""",
        database=target_database,
        workgroup=target_workgroup)    

    df_pivoted = pd.pivot_table(
        df_all,
        index=["product_brand_name", "zip_code", "year_month"],
        columns=["audience", "channel"],
        values=["reach", "engage", "cost"],
        aggfunc="sum"
    )

    df_pivoted.columns = ["_".join(col[::-1]).strip() for col in df_pivoted.columns]
    df_pivoted.reset_index(inplace=True)

    try:
        wr.s3.to_parquet(
            df=df_pivoted,
            path=f'{output_path}{today}/',
            dataset=True,
            database=target_database,
            table=output_table_name,
            mode='overwrite'
        )
    except Exception as e:
        raise e
final = generate_pivoted(target_database, target_workgroup, output_table_name, output_path, today)
final

In [ ]:
#Trigger post-pivot sql
current_path = str(Path.cwd())
current_path = current_path.replace('notebooks', 'sql')
final_path_post = Path(current_path + '/mars_post_pivot.sql')
trigger_all_functions([final_path_post])